In [1]:
from collections import Counter
import numpy as np
import pandas as pd

In [1]:
class DecisionTree:
    
    def __init__(self, criterion='gini', min_leaf=5, max_depth=3):
        
        self.tree = None
        self.min_leaf = min_leaf
        self.max_depth = max_depth
        self.criterion = criterion
    
    class Node:
        
        def __init__(self, index, t, true_branch, false_branch):
            
            self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
            self.t = t  # значение порога
            self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
            self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле
    
    class Leaf:
        
        def __init__(self, data, labels, criterion):
            
            self.data = data
            self.labels = labels
            self.criterion = criterion
            self.prediction = self.predict()
    
        def predict(self):
            
            if self.criterion == 'regression':
                return self.labels.mean()
            
            else:
                classes = Counter(self.labels)
                return max(classes, key=classes.get)
            
    # Расчет критерия информативности
    def impurity_criterion(self, labels, criterion):
        
        match criterion:
            case 'gini':
                classes = Counter(labels)
                impurity = 1
                for label in classes:
                    p = classes[label] / len(labels)
                    impurity -= p ** 2   
            
            case 'entropy':
                classes = Counter(labels)
                impurity = 0
                for label in classes:
                    p = classes[label] / len(labels)
                    impurity -= p * np.log2(p) if p != 0 else 0
        
            case 'regression':
                impurity = ((labels - labels.mean())**2).sum() / labels.shape[0]
        
        return impurity
    
    # Расчет качества
    def quality(self, left_labels, right_labels, criterion, current_criterion):
        
        # доля выборки, ушедшая в левое поддерево
        p = left_labels.shape[0] / (left_labels.shape[0] + right_labels.shape[0])
        
        return current_criterion - p * self.impurity_criterion(left_labels, criterion) - (1 - p) * self.impurity_criterion(right_labels, criterion)
    
    # Разбиение датасета в узле
    def split(self, data, labels, index, t):
        
        left = np.where(data.iloc[:, index] <= t)
        right = np.where(data.iloc[:, index] > t)

        true_data = data.iloc[left]
        false_data = data.iloc[right]
        true_labels = labels.iloc[left]
        false_labels = labels.iloc[right]
        
        return true_data, false_data, true_labels, false_labels
    
    # Нахождение наилучшего разбиения
    def find_best_split(self, data, labels, criterion, min_leaf):
        
        current_criterion = self.impurity_criterion(labels, criterion)
        
        best_quality = 0
        best_t = None
        best_index = None
    
        n_features = data.shape[1]
    
        for index in range(n_features):
            
            # будем проверять только уникальные значения признака, исключая повторения
            t_values = np.unique(data.iloc[:, index])

            for t in t_values:
                true_data, false_data, true_labels, false_labels = self.split(data, labels, index, t)
                
                #  пропускаем разбиения, в которых в узле остается менее min_leaf объектов
                if len(true_data) < min_leaf or len(false_data) < min_leaf:
                    continue
                    
                current_quality = self.quality(true_labels, false_labels, criterion, current_criterion)
                
                #  выбираем порог, на котором получается максимальный прирост качества
                if current_quality > best_quality:
                    best_quality, best_t, best_index = current_quality, t, index
        
        return best_quality, best_t, best_index
    
    # Построение дерева с помощью рекурсивной функции
    def build_tree(self, data, labels, criterion, min_leaf, max_depth):
        
        quality, t, index = self.find_best_split(data, labels, criterion, min_leaf)
    
        #  Базовый случай - прекращаем рекурсию, когда нет прироста качества
        #  или достигнут критерий останова

        if (max_depth == 0) or (quality == 0):
            return self.Leaf(data, labels, criterion)
        
        true_data, false_data, true_labels, false_labels = self.split(data, labels, index, t)
        
        # Рекурсивно строим два поддерева, обращая внимание на критерий останова
        true_branch = self.build_tree(true_data, true_labels, criterion, min_leaf, max_depth=max_depth-1)
        false_branch = self.build_tree(false_data, false_labels, criterion, min_leaf, max_depth=max_depth-1)
        
        # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
        return self.Node(index, t, true_branch, false_branch)
    
    def fit(self, X, y):
        self.tree = self.build_tree(X, y, criterion=self.criterion, min_leaf=self.min_leaf, max_depth=self.max_depth)
    
    def predict_object(self, obj, node):

        #  Останавливаем рекурсию, если достигли листа
        if isinstance(node, self.Leaf):
            answer = node.prediction
            return answer

        if obj[node.index] <= node.t:
            return self.predict_object(obj, node.true_branch)
        else:
            return self.predict_object(obj, node.false_branch)
    
    def predict(self, data):
        
        if self.criterion == 'regression':
            return np.array([self.predict_object(data.iloc[i, :], self.tree) for i in range(data.shape[0])])
        
        else:
            classes = []
            for obj in data:
                prediction = self.predict_object(obj, self.tree)
                classes.append(prediction)
            return classes